# Template Jupyter Python Notebook for FIESTA2017

FIESTA2017, <a href="http://t2.lanl.gov/fiesta2017" target="_new">t2.lanl.gov/fiesta2017</a> <br />
Sep. 17-22, 2017 <br />
Santa Fe, NM, USA

<a id="Abstract"></a> -- Invisible anchor here
### Abstract

# Welcome to FIESTA17!

<strong><i>The abstract for this notebook can be added here.</i></strong>

<p>In this mini-workshop, we will work on a very practical example of a modern nuclear data evaluation. In recent years, important efforts have been devoted to the evaluation of the Prompt Fission Neutron Spectrum (PFNS) and Prompt Fission Neutron Multiplicity (PFNM) for important actinides and for incident neutron energies from thermal up to 20 MeV. After briefly reviewing basic concepts on the physics of prompt fission neutrons, we will perform an evaluation of the PFNS in the neutron-induced fission reaction on Pu-239.</p>

<p>We will make use of modern evaluation codes, and use the framework provided by IPython Notebooks to deliver this course. Hands-on, step-by-step instructions will be provided to perform a complete evaluation,  including how to:</p>

<ul>
    <li>Retrieve experimental data from the Nuclear Data Center and IAEA servers</li>
    <li>Retrieve evaluated files from ENDF libraries</li>
    <li>Analyze differential experimental data and old publications</li>
    <li>How to deal with discrepant data sets</li>
    <li>Evaluate a covariance matrix</li>
    <li>Run a modern nuclear reaction code (e.g., CoH, TALYS, EMPIRE)</li>
    <li>Use Bayesian statistical inference tools</li>
    <li>Produce results in ENDF and GND (XML) formats</li>
</ul>
    
<p>Because of time constraints, for each item, we will work on one specific example, while more complete files and documentations will be provided separately.</p>

<p>During this course, we will introduce notions pertaining to: nuclear reaction mechanisms, statistical techniques, Bayesian inference, and XML format. A basic knowledge in these domains would be useful, but is not required. Some notions of the Python programming language would also be useful, but again, not required.</p>

<a id="SchoolLinks"></a>
### School Lecture Slides & Notes

Links here.
<a href="" target="_new">School lecture</

### Table of Contents

<ul>
<li><a href="#Abstract">Abstract</a></li>
<li><a href="#BeforeWeStart">Before we start</a></li>
<li><a href="#SchoolLinks">School Lecture Slides & Notes</a></li>
<li><a href="#PromptFissionNeutrons">Prompt fission neutrons</a></li>
<li><a href="#RetrievingEvaluatedData">Retrieving Data from Evaluated Libraries</a></li>
<li><a href="#RetrievingExperimentalData">Retrieving Experimental Data</a></li>
<li><a href="#AnalyzingExperimentalData">Analyzing Experimental Data</a></li>
<li><a href="#ModelCalculations">Model Calculations</a>
<ul>
<li><a href="#MadlandNix">Madland-Nix model</a></li>
<li><a href="#Maxwellian">Maxwellian spectrum</a></li>
<li><a href="#Watt">Watt spectrum</a></li>
</ul>
</li>
<li><a href="#Bayes">Learning the "Bayesian Way"</a></li>
<li><a href="#ENDFFormatting">Producing Formatted Evaluated Results</a></li>
<li><a href="#Processing">Processing for Applications</a></li>
<li><a href="#Benchmarking">Benchmarking and Feedback from Integral Data</a></li>
<li><a href="#FinalNotes">Final Notes</a></li>
</ul>

<a id="Abstract"></a>

<a id="BeforeWeStart"></a>
### Before we start

<h4><font color="darkred">IMPORTANT: Install python packages needed for this notebook</font></h4>

<a href="http://ipython.org/install.html">IP[y]: IPython Interactive Computing</a>

<ol>
<li>Download free edition of <a href="https://store.enthought.com/downloads/">Enthought's Canopy</a></li>
<li>Update IPython to the current version, using the Terminal: <font face="courier">enpkg ipython</font></li>
<li>Python 2.7 or >= 3.3 (if you use Python 2.6 or 3.2, you will need to use IPython 1.0)</li>
</ol>


### Setting environment and plotting variables, and importing Python libraries [DO NOT MODIFY]

In [1]:
import numpy as np # www.numpy.org :: numerical tools for scientific computing
import scipy as sp # www.scipy.org :: "ecosystem of open-source software for mathematics, science, and engineering"

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

import IPython as IP
from IPython.display import Image, FileLink, FileLinks, YouTubeVideo

%pylab inline

# settings for plots generated by matplotlib

import matplotlib as mpl
mpl.rcParams['lines.linewidth']=2
mpl.rcParams['font.size']=16

# define some plot styles

font = {'family' : 'serif',
        'color'  : 'darkred',
        'weight' : 'normal',
        'size'   : 20,
        }

Populating the interactive namespace from numpy and matplotlib


### Defining paths to image and data files

In [2]:
# Defining paths to image and data files

BASEDIR  = "/Users/talou/Sites/fiesta2017/jupyter/"  #-- TO ADJUST BY USER

IMAGEDIR = BASEDIR+"images/" # images
DATADIR  = BASEDIR+"data/"   # all data (experiment, ENDF, etc) files needed for this notebook
PAPERDIR = BASEDIR+"papers/" # publications, papers, reports, ...
OUTDIR   = BASEDIR+"output/" # output generated by the user of this notebook should be here

<a id="PromptFissionNeutrons"></a>

### Prompt Fission Neutrons

De-excitation of excited primary fission fragments by emission of "prompt" fission neutrons and photons.

<strong>The simple picture:</strong> A binary fission event produces two fragments, often a light and a heavy partner, that are deformed and excited compared to their ground-state configuration. They will emit so-called prompt neutrons and photons to get back to a more stable configuration. Prompt neutrons will remove most of the excitation energy, while $\gamma$ rays will tend to remove most of their angular momentum.

<strong>Not so simple...</strong> some neutrons may be emitted prior to scission, at scission, or during the acceleration of the two fragments. At higher incident energies, some neutrons may come from a pre-equilibrium contribution. Some anisotropy in the angular distribution of the neutrons can be due to emission of neutrons from rotating fragments, or/and from "scission" neutrons, etc.

We will not worry about these complications here.


In [3]:
Image(IMAGEDIR+"PostScissionCascade.png" , width="50%")

<strong>The question we want to address here is: what is the energy spectrum (distribution) of the emitted neutrons?</strong>

<p><font color="darkred" size="4em">The goal of an evaluation is to assess the value and accuracy of a physical quantity of interest to the best of our knowledge. This knowledge can come from experimental data, fundamental physics theory, modeling, and even "expert guesses".</font></p>

<p><font color="darkred" size="4em">The role of the evaluator is to collect all pertinent information and estimate the best value of this quantity, along with its uncertainty.</font></p>

<a id="RetrievingEvaluatedData"></a>

### Retrieving Data Files from Evaluated Libraries

#### From the ENDF-102 Manual:

<strong><font color="darkred" size="4em">ENDF :: Evaluated Nuclear Data Format</font></strong>

[ENDF Manual](papers/ENDF-102.pdf) // how to include a link to a local file

Many <strong>sub-libraries</strong>:

<ul>
<li>NSUB=0: photo-nuclear data</li>
<li>NSUB=4: radioactive decay data</li>
<li>NSUB=10: incident neutron data</li>
<li>NSUB=1,5,11: fission yields</li>
<li>...</li>
</ul>

#### The Anatomy of an ENDF file

###### MF=1, MT=451

###### MF=5, MT=18 (prompt fission neutron spectrum)

                                    5         18       4563          19437 1451 1355
    ...
                                   35         18       6373          09437 1451 1385

In [4]:
Image(IMAGEDIR+"MF5MT18screenshot.png", width="60%")

In [5]:
# Some ENDF routines here...

import re

# Retrieve a section (MF,MT) from an ENDF file

def readSection(filename,MFsearch,MTsearch):
    mysection=[]
    with open(filename) as f:
        lines = f.readlines()
        for line in lines:
            MAT=int(line[66:70])
            MF=int(line[70:72])
            MT=int(line[72:75])
            if (MF==MFsearch and MT==MTsearch):
                mysection.append(line)
    mysection = ''.join(mysection)
    return mysection

def readHEAD (s):
    ZA = int(addE(s[0:11]))
    AWR = float(addE(s[11:22]))
    L1 = int(s[22:33])
    L2 = int(s[33:44])
    N1 = int(s[44:55])
    N2 = int(s[55:66])
    return ZA,AWR,L1,L2,N1,N2

def readTAB1(s):
    C1 = float(addE(s[0:11]))
    C2 = float(addE(s[11:22]))
    L1 = int(s[22:33])
    L2 = int(s[33:44])
    NR = int(s[44:55])
    NP = int(s[55:66])
    MAT = int(s[66:70])
    MF = int(s[70:72])
    MT = int(s[72:75])

    # Exercise: implement reading TAB1 subsection
    return C1, C2, L1, L2, NR, NP

def readTAB2(s):
    # Exercise: implement reading TAB2 subsection
    return

def addE (s):
    s = re.sub('-','e-',s)
    s = re.sub('\+','e+',s)
    return float(s)


In [6]:
Image(IMAGEDIR+"exercise-icon.png", width="40px")

<strong>Exercise:</strong> implement reading TAB1 subsection

<strong>Exercise:</strong> implement reading TAB2 subsection

In [7]:
Image(IMAGEDIR+"exercise-icon.png", width="40px")

<strong>Exercise</strong> Modify script to retrieve JENDL-4.0 and JEFF-3.1 results, and compare them on a plot.

In [8]:
# open file 'pu239.pfns.endfb71' and plot results

neutronEnergies = [ 0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0, 6.0, 
                   7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 17.5, 20.0 ]
numberEnergies = len(neutronEnergies)

x = [[] for i in range(numberEnergies)]
y = [[] for i in range(numberEnergies)]

with open (DATADIR+'pu239.pfns.endfb71','r') as f:
    lines = f.readlines()
    
    i=-1
    for line in lines:
        if (line == '\n'):
            continue
        if (line[0]=='#'):
            i=i+1
            print("En = %6.2f MeV" % float(line[line.find(']')+1:].strip()))
            continue
        
        data=line.split()
        x[i].append(float(data[0]))
        y[i].append(float(data[1])*1e6)
    
xv = [[] for i in range(numberEnergies)]
yv = [[] for i in range(numberEnergies)]

for i in range(numberEnergies):
    xv[i]=np.array(x[i])
    yv[i]=np.array(y[i])


IOError: [Errno 2] No such file or directory: '/Users/talou/Sites/fiesta2017/jupyter/data/pu239.pfns.endfb71'

In [ ]:
# Checking data sets
i=640
print (xv[0][i], yv[0][i])

In [ ]:
# outgoing energy grid
nout=len(x[0])
egrid = np.empty(nout,dtype=float)
for i in range(nout):
    egrid[i] = x[0][i]

# Maxwellian
T=1.5
#y=2/sqrt(3.14159*T**3)*sqrt(x)*exp(-x/T)

Mw142 = np.empty(nout,dtype=float)
for i in range(nout):
    e=egrid[i]
    Mw142[i] = 2/sqrt(3.14159*T**3)*sqrt(e)*exp(-e/T)
    
# Compute ratios to Maxwellian
ratios = []
for i in range(numberEnergies):
    ratios.append(np.empty(nout))
    ratios[i] = np.divide(yv[i],Mw142)

In [ ]:
# plot results

fig = plt.figure(figsize=(16,16))

gs = gridspec.GridSpec(2,2)
#plt.subplots_adjust(hspace=0.5)

ax1 = plt.subplot(gs[0,:])
ax2 = plt.subplot(gs[1,0])
ax3 = plt.subplot(gs[1,1])

#ax1 = fig.add_axes([0.1,0.1,1.0,1.0])
ax1.set_xscale("log")
ax1.set_xlim(1e-3,20)
ax1.set_xlabel('Outgoing Neutron Energy (MeV)')
ax1.set_ylabel('Neutron Spectrum (1/MeV)')
ax1.set_ylim(0.0,0.4)
ax1.set_title('Pu-239 PFNS')
ax1.legend()
plt.legend(loc=1)

iskip = 10

# absolute value
for i in range(0,numberEnergies,iskip):
    ax1.plot(xv[i],yv[i],label=str(neutronEnergies[i])+" MeV")

#plt.legend(bbox_to_anchor=(1.05,1), loc=3, borderaxespad=0.)

# log-log scale
ax2.loglog()
ax2.set_xlim(1e-3,20)
ax2.set_ylim(1e-7,1)
for i in range(0,numberEnergies,iskip):
    ax2.plot(xv[i],yv[i],label=str(neutronEnergies[i])+" MeV")

# ratio to Maxwellian
ax3.semilogx()
ax3.set_xlim(1e-3,20)
ax3.set_ylim(0.5,1.2)
for i in range(0,numberEnergies,iskip):
    ax3.plot(xv[i],ratios[i],label=str(neutronEnergies[i])+" MeV")

ax1.legend(prop={'size':10})
ax2.legend(prop={'size':10})
ax3.legend(prop={'size':10})

plt.show()

In [ ]:
Image(IMAGEDIR+"attention-icon.png", width="40px")

Checking what already exists in the different libraries for a particular physical quantity of interest is one of the first things to do. But sometimes, no prior evaluation exists at all, e.g., multiplicity-dependent neutron spectra, not even the proper ENDF format...

<a id="RetrievingExperimentalData"></a>

### Retrieving Experimental Data

Some <strong>nuclear data centers</strong> to retrieve data from:
<ul>
    <li><a href="http://www.nndc.bnl.gov/exfor/exfor.htm" target="new">National Nuclear Data Center, Brookhaven</a> National Laboratory, USA</li>
    <li><a href="http://www.oecd-nea.org/janisweb/search/exfor" target="new">Nuclear Energy Agency, Paris</a>, France</li>
    <li><a href="http://www-nds.iaea.org" target="new">International Atomic Energy Agency, Vienna</a>, Austria</li>
</ul>

In [ ]:
# not working... I am not sure why...

import IPython as IP
url = "http://www.google.com"
iframe = '<iframe src=' + url + 'width=700 height=350></iframe>'
IP.display.HTML(iframe)
#from IPython.display import IFrame
#IFrame(url, width=700, height=350)


In [ ]:
Image(IMAGEDIR+"exercise-icon.png", width="40px")

<strong>Exercise:</strong> Try to retrieve the same information from the NEA Databank website.

In [ ]:
# Staples'1995 PFNS data (EXFOR# 13982-003)
FileLink(DATADIR+'pu239.pfns.Staples95.dat')

In [ ]:
# Starostov'1985 PFNS data (EXFOR# 40930-008)
FileLink(DATADIR+'pu239.pfns.Starostov85.dat')

In [ ]:
Image(IMAGEDIR+"exercise-icon.png", width="40px")

<strong>Exercise:</strong> replace Staples by Starostov or Knitter data


In [ ]:
from re import match

f = open(DATADIR+'pfns.dat','r')
lines = f.readlines()
f.close()

for line in lines:
    if match(r'ENTRY',line):
        EXFOR = line.split()[1]
    if match(r'AUTHOR',line):
        FirstAuthor = line[line.index('(')+1:line.index(',')]
    if match(r'REFERENCE',line):
        Reference = line.split()[1]

print ('EXFOR entry:  {0}'.format(EXFOR))
print ('First Author: {0}'.format(FirstAuthor))
print ('Reference:    {0}'.format(Reference))

In [ ]:
# Find the DATA section
iline=0
for i,line in enumerate(lines,0):
    if line.split():
        if (line.split()[0]=='DATA'):
            iline = i
            
# Extract DATA values into a numpy array
data=[]
for line in lines[iline:]:
    if (line[:7]=='ENDDATA'):
        break
    fields = line.split()
    try:
        raw_data = [float(x) for x in fields]
        data.append(raw_data)
    except:
        continue
    
data=array(data)

# Organize data for different incident neutron energies

Einc = [0.5,1.5,2.5,3.5]

expdata = []
for i in range(size(Einc)):
    print (i, "-", Einc[i], "MeV")
    mask = data[:,0]==Einc[i]
    raw_data = data[mask]
    expdata.append(raw_data)

In [ ]:
#print expdata[0][:,1]

In [ ]:
# Plot experimental data

fig=figure(figsize=(14,14))
fig.suptitle('PFNS Experimental Data from {0}, EXFOR Entry # {1}'.format(FirstAuthor,EXFOR),fontdict=font)

ax = plt.subplot(2,2,1,xlabel="Outgoing neutron energy (MeV)", ylabel="Neutron Spectrum (1/MeV)")
for i in range(size(expdata)):
    plt.loglog(expdata[i][:,1],expdata[i][:,2],marker='o',label='{0} MeV'.format(Einc[i]))
plt.legend()

ax = plt.subplot(2,2,2,xlabel="Outgoing neutron energy (MeV)", ylabel="Neutron Spectrum (1/MeV)")
for i in range(size(expdata)):
    plt.plot(expdata[i][:,1],expdata[i][:,2],marker='o',label='{0} MeV'.format(Einc[i]))
plt.legend()

ax = plt.subplot(2,2,3,xlabel="Outgoing neutron energy (MeV)", ylabel="Neutron Spectrum (1/MeV)")
plt.plot(expdata[0][:,1],expdata[0][:,2],'r.',label='{0} MeV'.format(Einc[0]))
plt.legend()

ax = plt.subplot(2,2,4,xlabel="Outgoing neutron energy (MeV)", ylabel="Neutron Spectrum (1/MeV)")
plt.errorbar(expdata[3][:,1],expdata[3][:,2],expdata[3][:,3],color='r',fmt='o',label='{0} MeV'.format(Einc[3]))
plt.legend()

plt.tight_layout()
plt.subplots_adjust(top=0.92)
plt.show()

In [ ]:
Image(IMAGEDIR+"exercise-icon.png", width="40px")

<strong>Exercise:</strong> perform similar retrieval and plotting tasks with Knitter data.

<a id="AnalyzingExperimentalData"></a>

### Analyzing Experimental Data, Uncertainties and Covariance Matrices

In [ ]:
Image(IMAGEDIR+"attention-icon.png", width="40px")

<font color="red">Experimental data present in the EXFOR database are <strong>rarely</strong> in a form suitable for automatic use for an evaluation, and careful analysis of the reference publication(s) is warranted.</font>

#### Constructing a Covariance Matrix for Experimental Data

Experimental Covariance Matrix for Knitter data

A realistic evaluation of an experimental covariance matrix requires a careful analysis of the experimental setup, data, publications, etc. This was done in Neudecker et al., LA-UR-13-24743 (2013) in the case of Pu-239 PFNS.

In the case of Knitter data, and only considering a very simplified version of Neudecker's analysis, we can consider different sources of uncertainty:

<ul>
<li>Statistical, $\sigma_s(E_i)$</li>
<li>Detector efficiency, $\sigma_d(E_i,E_j)$</li>
<li>Multiple scattering corrections, $\sigma_m(E_i,E_j)$</li>
<li>Deconvolution uncertainty (related to energy resolution), $\sigma_e(E_i,E_j)$</li>
</ul>

to reconstruct a covariance matrix:

\begin{eqnarray}
cov\left(N(E_i),N(E_j)\right) \equiv cov\left(N_i,N_j\right) &=& \sigma_s^2 \delta_{ij} + \sigma^d_i\sigma^d_j\rho^d_{ij}+\sigma^m_i\sigma^m_j\rho^m_{ij}+\sigma^e_i\sigma^e_j\rho^e_{ij}
\end{eqnarray}

In each case, correlations are assumed to decrease with increasing distance between two energy points, and are obtained as:

\begin{eqnarray}
\rho_{ij} = exp\left[-\left(\frac{(E_i-E_j)}{max(E_i,Ej)}\right)^2\right]
\end{eqnarray}





In [ ]:
FileLinks(DATADIR+"KnitterStaples/")

In [ ]:
# Build covariance matrix for Knitter data

# load all sources of uncertainty
energies, statistical, energyUnc, multipleScattering, deconvolution, detectorEfficiency = np.loadtxt(
DATADIR+"KnitterStaples/KnitterPartialRelUnc.dat",comments='#',unpack=True)

# load PFNS and total relative uncertainty
Eout, PFNS, TotalRelUnc = np.loadtxt(DATADIR+"KnitterStaples/KnitterTotalDataRelUnc.dat",comments="#",unpack=True)

ne=Eout.size

# load correlation matrix
KnitterCorr = np.zeros([ne,ne])
with open(DATADIR+"KnitterStaples/KnitterTotalCor.dat","r") as f:
    for i in range(ne):
        f.readline()
        for j in range(ne):
            dummy, dummy, KnitterCorr[i][j] = f.readline().split()


In [ ]:
# multiple scattering corrections

sigms = np.zeros([ne])
covms = np.zeros([ne,ne])
corms = np.zeros([ne,ne])

for i in range(ne):
    if (energies[i]<0.25):
        sigms[i]=21.0
    elif (energies[i]>2.0):
        sigms[i]=0.0
    else:
        sigms[i]=21.0-21.0*(energies[i]-0.25)/(2.0-0.25)

for i in range(ne):
    for j in range(ne):
        if (i==j):
            covms[i][j]=sigms[i]**2
        else:
    #        covms[i][j]=0.0
            covms[i][j]=sigms[i]*sigms[j]*exp(-((energies[i]-energies[j])/max(energies[i],energies[j]))**2)

for i in range(ne):
    for j in range(ne):
        if (covms[i][i]==0.0 or covms[j][j]==0.0):
            corms[i][j]=1.0
        else:
            corms[i][j]=covms[i][j]/sqrt(covms[i][i]*covms[j][j])


In [ ]:
fig=figure(figsize(12,6))

gs = gridspec.GridSpec(1,2)

ax1 = plt.subplot(gs[0,0])
ax2 = plt.subplot(gs[0,1])

ax1.axis([0,2.0,-1,25])
ax1.set_xlabel("Neutron Energy (MeV)")
ax1.set_ylabel("Relative Uncertainty (%)")
ax1.plot(energies,sigms)

ax2.axis([0.25,2.0,0.25,2.0])
ax2.set_aspect('equal')
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel("Neutron Energy (MeV)")
ax2.set_ylabel("Neutron Energy (MeV)")
ax2.pcolormesh(energies,energies,corms)

plt.tight_layout()
plt.show()

In [ ]:
Image(IMAGEDIR+"exercise-icon.png", width="40px")

<strong>Exercise:</strong> obtain the covariance matrices corresponding to the other sources of uncertainty.

In [ ]:
# plot all sources of uncertainty

fig=figure(figsize=(18,6))
fig.suptitle('Knitter data, Uncertainties & Correlations',fontdict=font)

gs = gridspec.GridSpec(1,3)

ax1 = plt.subplot(gs[0,0])
ax2 = plt.subplot(gs[0,1],sharex=ax1)
ax3 = plt.subplot(gs[0,2])

ax1.axis([0,14,1,200])
ax1.set_xlabel("Neutron Energy (MeV)")
ax1.set_ylabel("Relative Uncertainty (%)")

ax1.semilogy(energies,statistical,'b--',linewidth=2,label="Statistics")
ax1.semilogy(energies,energyUnc,'r:',linewidth=2,label="Energy Uncertainty")
ax1.semilogy(energies,multipleScattering,'g-',linewidth=2,label="Multiple Scattering")
ax1.semilogy(energies,deconvolution,'g-.',linewidth=2,label="Deconvolution")
ax1.semilogy(energies,detectorEfficiency,'r--',linewidth=2,label="Detector Efficiency")
ax1.semilogy(energies,TotalRelUnc,'r-',linewidth=3)

ax2.set_xlabel("Neutron Energy (MeV)")
ax2.set_ylabel("Relative Uncertainty (%)")

ax2.set_yscale('log')
ax2.grid()
ax2.errorbar(Eout,PFNS,TotalRelUnc*PFNS/100,fmt='o')
ax2.plot(Eout,PFNS,'r-')

ax3.set_xlabel('Neutron Energy (MeV)')
ax3.set_ylabel('Neutron Energy (MeV)')

ax3.set_xscale('log')
ax3.set_yscale('log')
ax3.set_aspect('equal')
ax3.axis([0.2,20,0.2,20])
ax3.pcolormesh(Eout,Eout,KnitterCorr)

ax1.legend(loc="best",prop={'size':12})
plt.tight_layout()
plt.subplots_adjust(top=0.85)
plt.show()

In [ ]:
Image(IMAGEDIR+"exercise-icon.png", width="40px")

<strong>Exercise:</strong> "toy" with sources of uncertainties and study the changes in the covariance matrix.

<a id="ModelCalculations"></a>

### Model Calculations

<a id="MadlandNix"></a>

#### Madland-Nix model

<a href="http://www.ans.org/pubs/journals/nse/a_20086">D.G.Madland and J.R.Nix, Nucl. Sci. Eng. 81, 213 (1982)</a>

The neutron energy spectrum $N(E)$, in the laboratory frame, for a fission fragment moving with a kinetic energy per nucleon $E_f$ reads

\begin{eqnarray}
N(E)=\frac{1}{2\sqrt{E_f}T_m^2}\int_{\left(\sqrt{E}-\sqrt{E_f}\right)^2}^{\left(\sqrt{E}+\sqrt{E_f}\right)^2}d\epsilon\sigma_c(\epsilon)\sqrt{\epsilon} \nonumber \\
\times \int_0^{T_m}{dTk(T)T \mbox{exp}(-\epsilon/T)},
\end{eqnarray}

where $\epsilon$ is the center-of-mass neutron energy, $k(T)$ is the temperature-dependent normalization constant given by

\begin{eqnarray}
k(T)^{-1}=\int_0^\infty{d\epsilon'\sigma_c(\epsilon')\mbox{ }\epsilon'\mbox{ }\mbox{exp}(-\epsilon'/T)},
\end{eqnarray}

$\sigma_c(\epsilon')$ is the energy-dependent cross section for the inverse process of compound nucleus formation, and $T_m$ is the maximum value of the triangular distribution of temperatures used to describe the excitation energy distribution in the fragments.

Considering the most probable fragmentation only, the average neutron energy spectrum in the laboratory frame is given by an average over the spectra for the light and heavy fragments 

\begin{eqnarray}
N(E) = \frac{1}{2}\left[ N_L(E,E^L_F,\sigma_c^L)+N_H(E,E^H_F,\sigma_c^H) \right],
\end{eqnarray}

assuming an equal number of neutrons emitted from the light and heavy fragments.

At higher incident energies, these formulas can be generalized to take into account the onset of multi-chance fission, i.e., the probability that one or more neutrons are emitted before the residual compound nucleus fissions. However, in the present work, we are only concerned with low-energy (0.5 MeV) fission for which only the first-chance fission component is non-zero.



<strong>Note</strong>: that's where it would have been nice to use my code...

<a id="Maxwellian"></a>

#### Maxwellian Approximation

At low incident neutron energies, a PFNS resembles a Maxwellian distribution at a given temperature $T$. This distribution is given by:

\begin{eqnarray}
\mathcal{M}_T(E) = \frac{2}{\pi T^3} \sqrt{E}\mbox{ exp}(-E/T)
\end{eqnarray}

In [ ]:
def Maxwellian(x,T):
    return 2.0/(pi*T**3)*sqrt(x)*exp(-x/T) 

x=linspace(0.0, 20.0, 200.)

T = ([1.20,1.42,1.50,1.80]) # temperatures [MeV]

y0 = Maxwellian(x,T[0])
y1 = Maxwellian(x,T[1])
y2 = Maxwellian(x,T[2])

fig=figure(figsize=(14,6))

fig.suptitle('Maxwellian Functions',fontdict=font)

ax1=subplot(1,2,1,xlabel="Outgoing neutron energy (MeV)", ylabel="Neutron Spectrum (1/MeV)")
plt.plot(x,y0,'r-',label='T={0} MeV'.format(T[0]))
plt.plot(x,y1,'b--',label='T={0} MeV'.format(T[1]))
plt.plot(x,y2,'g-.',label='T={0} MeV'.format(T[2]))
plt.legend()

ax2=subplot(1,2,2,xlabel="Outgoing neutron energy (MeV)", ylabel="Neutron Spectrum (1/MeV)")
plt.loglog(x,y0,'r-',label='T={0} MeV'.format(T[0]))
plt.loglog(x,y1,'b--',label='T={0} MeV'.format(T[1]))
plt.loglog(x,y2,'g-.',label='T={0} MeV'.format(T[2]))
plt.loglog(x,Maxwellian(x,T[3]),'m-',label='T={0} MeV'.format(T[3]))

plt.legend()
plt.show()



<a id="Watt"></a>

#### Watt Spectrum

In [ ]:
Image(IMAGEDIR+'Watt-PhysRev1.png')

In [ ]:
Image(IMAGEDIR+'Watt-PhysRev2.png')

In [ ]:
# Code Watt spectrum here, with (a,b) parameters that can be tuned
def Watt(x,a,b):
    return exp(-a*x)*sinh(sqrt(b*x)) # no normalization!!

a=([1.0,2.0,3.0])
b=([0.5,3.0,4.0])

x=linspace(0., 20., 300)

y0=Watt(x,a[0],b[0])
y1=Watt(x,a[1],b[1])
y2=Watt(x,a[2],b[2])

fig=figure(figsize=(12,6))

fig.suptitle('Watt Spectra',fontdict=font)

ax1=subplot(1,2,1,xlabel="Outgoing neutron energy (MeV)", ylabel="Neutron Spectrum (1/MeV)")
plt.plot(x,y0,'r-',label='a={0} ; b={1}'.format(a[0],b[0]))
plt.plot(x,y1,'b--',label='a={0} ; b={1}'.format(a[1],b[1]))
plt.plot(x,y2,'g-.',label='a={0} ; b={1}'.format(a[2],b[2]))
plt.legend()

ax2=subplot(1,2,2,xlabel="Outgoing neutron energy (MeV)", ylabel="Neutron Spectrum (1/MeV)")
plt.loglog(x,y0,'r-',label='a={0} ; b={1}'.format(a[0],b[0]))
plt.loglog(x,y1,'b--',label='a={0} ; b={1}'.format(a[1],b[1]))
plt.loglog(x,y2,'g-.',label='a={0} ; b={1}'.format(a[2],b[2]))

plt.legend(loc=3)
plt.show()


<a id="Bayes"></a>

### Learning from Model Calculations and Experimental Data the "Bayesian Way"

##### A few quotes

In [ ]:
Image(IMAGEDIR+'Rodin_Le_Penseur.jpg', width="150")

<font color="darkred">"Probability theory is nothing but common sense reduced to calculation,"</font> Pierre-Simon Laplace, 1819

<font color="darkred">"Ignorance is preferable to error and he is less remote from the truth who believes nothing than he who believes what is wrong."</font> Thomas Jefferson, 1781

<font color="darkred">"Statistics are like a drunk with a lampost: used more for support than illumination."</font> Winston S. Churchill, 1874-1965

In [ ]:
paper=Image(IMAGEDIR+'BayesPaperScreenshot.png')
portrait=Image('images/Bayes.jpg',width="200px")
display(paper,portrait)

In [ ]:
YouTubeVideo('0F0QoMCSKJ4') # Introduction to Bayesian Statistics on YouTube

<strong>Bayes's theorem</strong> is a rather simple consequence of basic rules of conditional probability theory:

\begin{equation*}P(A|BC)=\frac{P(B|AC)P(A|C)}{P(B|C)} \end{equation*}

The <i>posterior</i> probability of an hypothesis $A$ given that both $B$ and $C$ are true is given by the product of a <i>likelihood</i> function, $P(B|AC)$, and a <i>prior</i> probability, $P(A|C)$.

If $B$ represent some experimental data that have been measured and $A$ is a physics model that represents the same physical quantity, $P(A|BC)$ represents the probability that the model $A$ is correct, given the observed $B$, under circumstances $C$.

#### The Kalman Filter

R.E.Kalman, <a href=PAPERDIR+"Kalman1960.pdf">"A New Approach to Linear Filtering and Prediction Problems"</a>

(First used in the Apollo space program for tracking trajectories.)

The time evolution of a physical state can be written as

\begin{equation}
x_k=F_kx_{k-1}+B_ku_k+w_k,
\end{equation}
where
<ul>
<li>$x_k$ represents the "true" state at time $k$</li>
<li>$F_k$ is the state transition model</li>
<li>$B_k$ is the control input model</li>
<li>$u_k$ is the control vector</li>
<li>$w_k$ is the noise, with $w_k\sim N(0,Q_k)$</li>
</ul>

This information is combined with observations

\begin{equation}
z_k=H_kx_k+v_k,
\end{equation}
with
<ul>
<li>$H_k$ is the observation model</li>
<li>$v_k$ is the observation noise, such that $v_k\sim N(0,R_k)$
</ul>

The Kalman filter technique contains two distinct phases: <strong>Predict</strong> and <strong>Update</strong>

<u>Predict</u>
\begin{eqnarray}
x_{k|k-1} & = & F_kx_{k-1}+B_ku_k \\
P_{k|k-1} & = & F_kP_{k-1}F_k^T+Q_k
\end{eqnarray}

<u>Update</u>
\begin{eqnarray}
y_k &=& z_k-H_kx_{k|k-1}\\
S_k &=& H_kP_{k|k-1}H_k^T+R_k \\
K_k &=& P_{k|k-1}H_k^TS_k^{-1} \mbox{(optimal Kalman gain)} \\
x_k &=& x_{k|k-1}+K_ky_k  \mbox{(updated state)}\\
P_k &=& \left( I-K_kH_k\right)P_{k|k-1} \mbox{(updated covariance)}
\end{eqnarray}




#### The Kalman Filter and Nuclear Data Evaluations

<ul>
<li>No dynamics: $u_k$, $B_k$ and $F_k$ disappear</li>
<li>No prediction step: $x_{k|k-1}=x_k$ and $P_{k|k-1}=P_k$</li>
</ul>

<p>
<font color="darkred">The Kalman filter is used as a <strong>recursive Bayesian estimation</strong>.</font></p>

<p>We are left with

\begin{eqnarray}
x_k &=& x_{k-1}+K_k\left(z_k-H_kx_{k-1}\right) \\
P_k &=& \left(I-K_kH_k\right)P_{k-1}
\end{eqnarray}
where

\begin{eqnarray}
H_k &=& \frac{\partial z_k}{\partial x_k} \\
K_k &=& P_{k-1}H_k^T\left( H_kP_{k-1}H_k^T+R_k\right)^{-1}
\end{eqnarray}
</p>

<h5>Extended Kalman filter</h5>

<p>Linear approximation with a first-order Taylor series expansion:</p>

\begin{equation*}
y=f(x)+e\simeq f(x_0)+C(x-x_0)+e,
\end{equation*}
where $C$ is the sensitivity matrix.

In the notation of <a href="http://www.ans.org/pubs/journals/nse/a_2021">T.Kawano, Nucl. Sci. Eng. <strong>131</strong>, 107 (1999)</a>, we get

\begin{eqnarray}
x_1 &=& x_0+XC^T\left(CXC^T+V\right)^{-1}\left( y-f(x_0)\right) \\
P &=& X-XC^T\left(CXC^T+V\right)^{-1}CX,
\end{eqnarray}
where $(x_0,X)$, respectively $(x_1,P)$, are the <i>prior</i>, respectively <i>posterior</i>, value and covariance matrix.



<strong>Exercise</strong> implement these equations in python.








In [ ]:
# Python implementation of the Kalman filter

# Goal: fit experimental PFNS data with a Watt spectrum

# x-grid
n=20
egrid = np.arange(n)


In [ ]:
# Watt spectrum, depends on two parameters (a,b)
def Watt(x,a,b):
    return exp(-a*x)*sinh(sqrt(b*x))

# default (prior) parameters
a0=1.0
b0=0.5

# prior vector & covariance matrix (20% unc; no correlation)
p0 = (a0,b0)
P0 = ([0.02*a0,0.0],[0.0,0.02*b0])

# prior Watt spectrum
y0=Watt(x,a0,b0)


In [ ]:
print (p0, P0)

In [ ]:
# plot -----------------------------------------------------------------
x=np.arange(0., 10.,0.01)

fig=figure(figsize=(8,6))

# prior
x0=x
y0=Watt(x,a0,b0)

plt.yscale('log')
plt.plot(x0,y0,'r-',label='prior; a={0} ; b={1}'.format(a0,b0))

plt.legend()
plt.show()

In [ ]:
# Measurements
# generate some arbitrary data (following a Watt spectrum with different parameters) with
# some noise
nd=10 # number of data points

edata=np.zeros(nd)   # energies
sdata=np.zeros(nd)   # spectrum values
uncdata=np.zeros(nd) # uncertainties on spectrum values

# "experimental" data loosely following a Watt spectrum with parameters (1.2,1.0)
def yd(x):
    return (1.0+np.random.normal(0,0.2))*Watt(x,1.2,1.0)

np.random.seed()
noise = np.random.normal(0,0.2,nd) # mean, standard deviation, number of samples

shape(edata)
for i in range(nd):
    edata[i]   = i+0.1
    sdata[i]   = yd(edata[i])
    uncdata[i] = (0.10+0.02*edata[i])*sdata[i]
#    uncdata[i] = 0.10*sdata[i]
    
# "experimental" covariance matrix
V=np.ndarray(shape=(nd,nd))
for i in range(nd):
    for j in range(nd):
        if (i==j):
            V[i,j]=uncdata[i]**2
        else:
            V[i,j]=0.0 # no correlation


In [ ]:
# plot -----------------------------------------------------------------
x=np.arange(0., 10.,0.01)

fig=figure(figsize=(8,6))

plt.yscale('log')
plt.plot(x0,y0,'r-',label='prior; a={0} ; b={1}'.format(a0,b0))
plt.errorbar(edata,sdata,yerr=uncdata,fmt='o',label='data')
#plt.plot(x,Watt(x,1.2,1.0),'b--')

plt.legend()
plt.show()

In [ ]:
# Design (D) and Sensitivity (C) matrices
# (here, the design matrix is the identity.)

# y-f(x0)
diff=np.zeros(nd)
for i in range(nd):
    diff[i] = sdata[i]-Watt(edata[i],a0,b0)

# Sensitivity matrix C

C = np.zeros(shape=(nd,2)) # 'nd' data points and '2' model parameters

delta = 0.05
for i in range(nd):
    x=edata[i]
    C[i,0] = (Watt(x,a0*(1+delta),b0)-Watt(x,a0,b0))/(delta*a0)
    C[i,1] = (Watt(x,a0,b0*(1+delta))-Watt(x,a0,b0))/(delta*b0)


In [9]:
C, diff

NameError: name 'C' is not defined

In [ ]:
# Kalman Filter Equations
#
# p1 = p0 + P0 C^T ( C P0 C^T + V)^-1 ( y - f(x0) ) # posterior parameter values
# P1 = P0 - P0 C^T ( C P0 C^T + V)^-1 C P0          # posterior parameter covariance matrix
#

Q = np.dot(np.dot(P0,transpose(C)),inv(np.dot(C,np.dot(P0,transpose(C)))+V)) # P0 C^T ( C P0 C^T + V)^-1 

p1 = p0 + np.dot(Q,diff)
P1 = P0 - np.dot(Q,np.dot(C,P0))



In [ ]:
p1, P1

In [ ]:
# plot posterior

x=np.arange(0., 10.,0.01)

fig=figure(figsize=(8,6))

plt.yscale('log')
plt.plot(x,y0,'r-',label='prior; a={0} ; b={1}'.format(a0,b0))
plt.errorbar(edata,sdata,yerr=uncdata,fmt='o',label='data')
plt.plot(x,Watt(x,1.2,1.0),'b--',label='"true" values')
plt.plot(x,Watt(x,p1[0],p1[1]),'g-',label='posterior')

plt.legend(loc=3)
plt.show()

<a id="ENDFFormatting"></a>

<a id="Processing"></a>

### Processing for Applications

Why do we need processing?

The ENDF evaluated nuclear data files have to be processed through a code like NJOY (http://t2.lanl.gov/nis/codes.shtml) before they can be used in a wide range of applications.

(no time to cover the processing part here.)

<a id="Benchmarking"></a>

### Benchmarking and Feedback from Integral Data

Talou et al., "Uncertainty Quantification of Prompt Fission Neutron Spectrum for n(0.5 MeV)+Pu-239," <a href="http://www.ans.org/pubs/journals/nse/a_10914">Nucl. Sci. Eng. 166, 254 (2010)</a>

In [ ]:
FileLink(DATADIR+"pfns.covmat.dat")

In [ ]:
# read PFNS covariance matrix from earlier 2010 study

f = open(DATADIR+"pfns.covmat.dat")
ne = int(f.readline()) # number of energy points
energies = np.empty(ne)
spectrum = np.empty(ne)
for i in range(ne):
    (energies[i],spectrum[i]) = f.readline().split()

scaling = 1.0

covmat = np.zeros([ne,ne])
for i in range(ne):
    if (i>0):
        f.readline() # skip one line
    for j in range(ne):
        covmat[i][j] = float(f.readline())*scaling**2

cormat = np.zeros([ne,ne])
for i in range(ne):
    for j in range(ne):
        cormat[i][j] = covmat[i][j]/sqrt(covmat[i][i]*covmat[j][j])

In [ ]:
# plot the covariance and correlation matrices

import matplotlib.gridspec as gridspec

plt.figure(figsize=(15,15))

gs = gridspec.GridSpec(2,2)

ax1 = plt.subplot(gs[0,0],aspect="equal")
ax2 = plt.subplot(gs[0,1],aspect="equal")

# correlation matrix
ax1.set_xscale('log')
ax1.set_xlim([0.001,20])
ax1.set_xlabel('Neutron Energy (MeV)')

ax1.set_yscale('log')
ax1.set_ylim([0.001,20])
ax1.set_ylabel('Neutron Energy (MeV)')

ax1.pcolormesh(energies,energies,cormat,cmap="afmhot")
ax1.set_title('Correlation Matrix')

# covariance matrix
ax2.set_xscale('log')
ax2.set_xlim([0.001,20])
ax1.set_xlabel('Neutron Energy (MeV)')

ax2.set_yscale('log')
ax2.set_ylim([0.001,20])
ax1.set_ylabel('Neutron Energy (MeV)')

ax2.pcolormesh(energies,energies,covmat,cmap="afmhot")
ax2.set_title('Covariance Matrix')

plt.show()

In [ ]:
# [1] Diagonalize the PFNS covariance matrix
# [2] Create random Gaussian samples around sqrt(eigenvalues)
# [3] Create samples of PFNS

s = linalg.eig(covmat) # [1] diagonalize here
eigenvalues  = s[0]
eigenvectors = s[1]

# print ne, eigenvalues.size (should be equal)
ne = eigenvalues.size

# [2] sample eigenvalue distributions using Gaussian distributions with standard deviations of sqrt(eigenvalues)
ns=1000

# [3] sample PFNS distributions
samples = np.empty([ns,ne])
eps = np.empty([ne])
for i in range(ns): # number of desired samples
    for j in range(ne):
        eps[j] = np.random.normal(0.0, sqrt(abs(eigenvalues[j])))
#        print j, eps[j]
    for j in range(ne): # corresponds to number of points on outgoing energy grid
        samples[i][j]=spectrum[j]+sum(eigenvectors[j]*eps)


In [ ]:
plt.figure(figsize=(10,8))

for i in range(ns):
    plt.plot(energies,samples[i],'c-',linewidth=0.25)

plt.plot(energies, spectrum, 'r-',linewidth=3.0)

plt.title('Sampled Prompt Fission Neutron Spectra')
plt.xlabel("Kinetic Neutron Energy (MeV)")
plt.ylabel("Spectrum (1/MeV)")
plt.axis([0,10,0,0.5])
plt.show()

In [ ]:
# plot histogram of mean energies from sampled spectra

from scipy.stats import norm
import matplotlib.mlab as mlab

# energy steps on the outgoing energy grid
deltaE = np.zeros(ne)
for i in range(ne-1):
    deltaE[i]=energies[i+1]-energies[i]

# calculate mean energies
meanEnergies = np.zeros(ns)
for i in range(ns):
    meanEnergies[i] = sum(samples[i]*energies[:]*deltaE[:])

# plot
plt.figure(figsize=(6,6))

plt.ylabel("Counts")
plt.xlabel("Average Neutron Energy (MeV)")
plt.hist(meanEnergies,50)

plt.show()

#### The "Sandwich Rule"

To quickly infer the impact of the uncertainty of the Pu-239 PFNS on an integral quantity such as the multiplication factor $k_{eff}$ for the Jezebel critical assembly, one can use the "sandwich rule":

\begin{eqnarray}
\sigma_{k_{eff}}=\mathbf{SCS^T}
\end{eqnarray}

where $\mathbf{S}$ is the sensitivity profile (vector) obtained as $\partial k_{eff}/\partial \chi(E)$, and $\mathbf{C}$ is the PFNS covariance matrix.

Databases of sensitivity profiles are being produced to quickly study the impact of an evaluation on integral data.

<a id="FinalNotes"></a>

# <center><font color="red">The End</font></center>

### Appendix A: Data Files

In [ ]:
FileLinks(DATADIR)

### Appendix B: Image Files

In [ ]:
FileLinks(IMAGEDIR)

### Appendix C: References on Prompt Fission Neutrons

<ul>
<li>J.Terrell, "Fission Neutron Spectra and Nuclear Temperatures", <a href="http://journals.aps.org/pr/abstract/10.1103/PhysRev.113.527">Phys. Rev. <strong>113</strong>, 527 (1959)</a></li>
<li>D.G.Madland and J.R.Nix, "New Calculation of Prompt Fission Neutron Spectra and Average Prompt Neutron Multiplicities," <a href="http://www.ans.org/pubs/journals/nse/a_20086">Nucl. Sci. Eng. <strong>81</strong>, 213 (1982)</a></li>
<li>B.E.Watt, "Energy Spectrum of Neutrons from Thermal Fission of U-235," <a href="http://journals.aps.org/pr/abstract/10.1103/PhysRev.87.1037">Phys. Rev. <strong>87</strong>, 1037 (1952)</a></li>
</ul>

### Appendix D: References on Bayesian Statistics

<ul>
<li>F.H.Froehner, <a href="http://www.fritz-froehner.de/logic01.htm">"The Logic of Scientific Data Evaluation"</a></li>
<li>R.E.Kalman, <a href=PAPERDIR+"Kalman1960.pdf">"A New Approach to Linear Filtering and Prediction Problems"</a></li>
</ul>

### Appendix E: Python Tools

<ul>
<li><a href="http://www.ipython.org">IPython</a>: interactive computing with Python</li>
<li><a href="http://www.numpy.org">NumPy</a>: numerical tools</li>
<li><a href="http://www.scipy.org">SciPy</a>: scientific tools</li>
<li><a href="http://www.matplotlib.org">Matplotlib</a>: plotting library</li>
</ul>

In [ ]:
FileLinks(PAPERDIR)